How to prune neurons?
- Maybe by replace activations with zero?
- Maybe with replace activations with random? What is random?

In [ ]:
from taker import Model
from taker.eval import evaluate_all
from taker.data_classes import PruningConfig, RunDataHistory
from taker.prune import prune_random
from tqdm import tqdm
import wandb

def run_random_replacement(m: Model, c: PruningConfig, sampled_text: str):
    # Prepare data logging
    history = RunDataHistory(c.datasets)
    wandb.init(
        project=c.wandb_project,
        entity=c.wandb_entity,
        name=c.wandb_run_name,
        )
    wandb.config.update(c.to_dict(), allow_val_change=True)

    # Get the random activations we will be replacing with
    preout_rand = m.get_attn_pre_out_activations(sampled_text)

    # evaluate the modified model
    if c.run_pre_test:
        data = evaluate_all(m, c.eval_sample_size,
            c.datasets, c.collection_sample_size)
        history.add(data)
        print(history.df.T)

    # run all the steps of pruning
    ff_pruned, attn_pruned = None, None
    for i in tqdm(range(c.n_steps)):
        # prune some neurons
        ff_pruned, attn_pruned, misc = \
            prune_random(m, attn_frac=c.attn_frac, ff_frac=c.ff_frac, ff_pruned=ff_pruned, attn_pruned=attn_pruned)

        # update the pruned neurons to be resampled
        preout_rand_masked = m.run_inverse_masking(preout_rand, "attn_pre_out")
        m.update_actadd(preout_rand_masked.reshape(m.cfg.n_layers, m.limit, m.cfg.d_model), "attn_pre_out")

        # evaluate the modified model
        data = evaluate_all(m, c.eval_sample_size,
            [c.focus], c.collection_sample_size)
        history.add(data)
        print(history.df.T)

    wandb.finish()
    return history


In [ ]:
random_input = """3KhCfVVkC5VNSchVtAyKurMGCHaMHS25SSKrJRD6yaNZSwSN3MsVBRNs6KmvQG3qYFntXDqxWtBKDcNHcDmHuRbNa39qWc7whK2LZPXKxUP8qWfmKfCEyfEzcUWJMDcaHve9qM7BGfurBeNCDryd7Ad2Rc2VYxymX4DGeuJZ8Thq4C3jjUAeAPpCn7wQZVeLx9A35JypdKTvgYGQKvfD6bgK3hVfq8tHhnDUWuTQppnBn4VTMFhYTK9A5dLHyqp9p5uCpD4uBzFJH2Yx4GGg5uwt6uCuVS2PfD239awAbXRh7euz3EzFVBC7eeSmeMz2Np5YEW8Ra5x26pqQvw897CRJjbmm3RxNcmYJqUFFwRPVAQrbVTHxfPQtf4R9NCJzfRGHRfNcQ9ZD9UapmNmfcXEKre8sBtyDnSpjET6nhDnqfdC79Qd4mSEQnSNyk2FKPkRbd4mpgyahgNTJh4ZcWY8SpVab8jMvBTA9yFmKJmtKTzcpvgdyy7HbbAyDZRztCuC5Jw5VLExnJgLX7Wath49LcL4qEJmC7w335MF9hGaYY9s7nQwbAzpARReC3EdfjVzxd7f2ru64f8Zb8KJr6JeUprrqyaQzfm9wtL2hFkxprUc5vGnJP3fghyNNeUFS3bsDEnbFVJaYGYnQBWeLtm74Fp4HMyhQYFXeVY4EFwSYMRCCLdd3pACADcZ5LjSrBTR9ryHMACxU5uhM7JTGRW9sAzzfM26W2vzuvGWjTFDsVfYWrAzvnm56C5wstVeTy9sEexYkCwag3MEGxr3qnJpQYESPZuMtEhhjeFTBnaMaZ2HJNHfEj2WkJXNGPjF4qsv48r3tEk462nDQ7UhTRcZvnDwAMSgFtv6PX2HXz8EqzCv2jgYUFvgajXjfmK9kzy9bjLaSyG6qSrSMWFWhLkwLWrednXdWFE822FCqNK8vsCxtpWpVCYK7C42gy7DFmFffPa53XWdT69UNefXumy5FGtqRasfrsA3yNr96uRSnBsjTSDyA7uLfxeWCcu87w7MXtQVXuNR9g9EXmNmLUR5nEWPjVsf2C8AdARb5q4VqafTZMYhxBUFcp2DNQf6Z7kFrJrC9kU4Lrmt4TUhr93EN28Wx4Mr6pGn8bqfvttLYW7KpWFxMwjHfnreh53ZBMHG477QV9qYmXYweN5uXe4Zd2Q94mGFuw9B2uzLsSqMGq9ZUGAfGtbmzn3ybn6dCbDwdQ8XhkBxJSHfTHxncNLEdPPkKvsjRRNUcBELxE48ChL36fUWWWcJtxYCuA9zvcMhFPHD8MBY3cX4qwN8vpcRCADM7g9px2J2ceK9MVTXEdJd93haQZG6E8eGLgbWt27jjR7rXYAaqm9uDuVLMgDLK9uMNhjAtsY89ymz9hvPeM7bGYKvCwfb8GujfKQAVw2JmRFHUQP4hBfDT9ccSG9Wd8mQUuGmVYVpEbUZfRQZ8wmqPkGQjSZZDU9XLWSzyNAgvZHXvpCbBYFzWtQPefsVsC6sD64TaqyHRmTxftS24QWmZzVMcemUhbfgZTxqukg7CGfNL4S4cHPFhV7n2hBxDXDNLRyYM4ZAQX5m9phCqPnTQw8T9wKwEhnm2pRLw2D748S89SSv5LYLUMXeVeG6EAZRtuyuDQbPJvRF3EK6zvJvqV5QtWVpkveQMU6xq757F97h5Af8vmM58t7k9U6GAr4RvFFZ6WnYkQVXE74MXw4HLWJwnCu3YevDaSNNJEeFqkmEGk3wnkrVyN8AKShTBLcrtw7qaWQktwdF9qRqF2Zf58ZC3suUg9B76TPD9nh3fFpMUuSQAwYzF8CH4zzRD2tV3F5zjxEKmnCpxdERCKgRSYX6KWRrnFwhuEp25vQaYUp4mCnXbgd6AVkUzVv64JnbC7Yxp23gpv5NYp6vfJCFuzRwaMbE6RRurKHP6FQ9j9KbSTx3CmgThn4Zfv39q2u6AfrBWm5sPpQQeh666FqFxGfAq7ttA8x8JPSMH5dh7EkjFhhxrQRBWMrSCHLMM3b9TzXKKANm2BruU43uW397dKw9vytSkbxS9AkAN6rbAxuY96SdwfN4vLHm7hU7nEDvkWt7Sn7jETEfSRTj9upFgxwKpn3nHkZ43zYCzWdthZRfN5m5WwdNYDgMbjAvZNCjgUyC6FT9nhMhJ7XChJG5q4u7Aa2PUMF6QNrqaMmuAEnZNdPCq7544ZdaW9vej59GqhUJnKHU9pfSwKbRt4wYV4de6JwmMkAFuawbnFXkYyCDCfmJAt3fJDNyuZpzXbPxreHgCQ6dSMqrD6P4w8nUK99eFHeLfhdUCy4AafDskbZrqb7P4ZWqMVPL8K2HqfEYSYMrfsXnyZ87RQ3fxSLMfym5SyzX9E8x3zKsvrRTWpGCbrYgVf3GszHUkCGuSkpfPW3xHkrknS73Rzt4DWjZTpx63C8zTPHyyuVTGqSFHkwa9McvSPtmK8v9VeU7HydWEdvxkxLvPhhYa2QwUqCQut4gRSXTjfRWv8r4ejkq3vwrjnEJDv2HNEf7yqm3gVR2Hq7Zrvm35QLghBDsKxJXEM9S8wFNrVTUAvjs9muaD42eDfqn6uCNKrYPTw44Q9gzHW5mEWfEtEWk2wj26FAzxZMmnvxuFsU3YucdbH2vXqB86dUWRucMqcjfg8FDBMDwdmcUrENNTUEdPtwhqSRN5hyWKPjGTxQSzuCx9VSpjn2k6LFzE3YKTLxXNz4UgTM4X98CDZkat6pFXwudGUucLFQgsh95g4BjbKTTHvCUfMfQhB8Qxuz2MkGtKSbAfNLFYd65Ky6fq6hkHqQZHFYFKu6jDdU3ZJup9zUSK49gDBDZnqQGgSraZkUKnTEdwVqAWaTWND28nXSCmzvUUYvmrYqZR6fVqqk7dcEFNduksGqqce8LXRhzesSGCmunpGWY4ewQYafK79yU7GMm8ZFnaZ4Ur2xA2PfDksu2MPJhv5X2fyNwRQPHhZjsQJJtedcx5Ktt7kuqYWxwt5SP5CpAbLNr3V4xGmMMCsrErrnXsS37decqMxW2TKxrTNJDUxvuMPaXAZ3zVRbVa72DQ2vcNBKx423Zn3GXcqUhkzaKEXhY79w4D7ejMrHhNeG7HJfwHzPDVpG5SQPZPK37AWHeu2rNjkArVYLRNXepDfwPhRX2jCKWSTQMxwcswNJEcsMhA8HEhvBpRUMktY9dh7mZpAfkT5VyVXSQh3Dns"""

c = PruningConfig(
    model_repo = "facebook/opt-1.3b",
    token_limit= 1000,
    dtype = "fp16",
    wandb_run_name = "opt-1.3b 5% resampling rand chars",
    wandb_project = "nicky-resampling-testing",
    focus = "pile",
    cripple="code",
    collection_sample_size = 1e4,
    eval_sample_size = 1e5,
    ff_frac = 0.00,
    attn_frac = 0.05,
    n_steps = 20,
)

m = Model(c.model_repo, limit=c.token_limit, dtype=c.dtype)
#run_random_replacement(m, c, random_input)

In [ ]:
import numpy as np

rand_image = np.random.randint(0, 255, [3, 244, 244])
print(m.processor(rand_image)["pixel_values"][0].shape)

c = PruningConfig(
    model_repo = "google/vit-base-patch16-224",
    token_limit= 1000,
    dtype = "fp32",
    wandb_run_name = "vit 10% resampling rand pixels",
    wandb_project = "nicky-resampling-testing",
    focus = "imagenet-1k",
    cripple="imagenet-1k-birds",
    collection_sample_size = 1e4,
    eval_sample_size = 1e4,
    ff_frac = 0.00,
    attn_frac = 0.10,
    n_steps = 10,
)

m = Model(c.model_repo, limit=c.token_limit, dtype=c.dtype)
run_random_replacement(m, c, rand_image)